<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Буду обращаться в мужском роде так как не указано имя в работе. Поздравляю тебя с первым твоим проектом и спасибо, что вовремя сдал задание:) Ты проделал большую работу. Далее в файле мои коммефнтарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

# Визуальный осмотр предоставленных данных
print(df.head(10))
print(df.info())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

Часть предоставленных данных находится в некорректном виде и их необходимо исправить.

Пропущены следующие значения:
- days_employed
- total_income

Некорректные значения:
- days_employed:
    - положительные значения превышают допустимый стаж.
    - отрицательные значения
- children
    - отрицательные значения
    
Данные не в едином стиле, что может привести к ошибкам в общих выводах:
- education:
        - разный регистр



<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошее начало, так же, для большего понимания данных можно было бы использовать метод `describe`


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# Проверяем количество и столбцы по пропущенным значениям
#print(df.isna().sum())

# Приведение значений столбца "education" к единому регистру
df['education'] = df['education'].str.lower()
# print(df['education'].unique())
# education_matching = df.groupby('education')['education_id'].unique().sort_values()
# print('\n', education_matching)

# уточняем, насколько положительные значения корректны через сравнение количества дней при стаже в 60 непрерывных лет (21 900)
# df['days_employed'] = df['days_employed'].replace([df['days_employed'] > 21900], '0')
df.loc[(df['days_employed'] > 21900), 'days_employed'] = 0
# print('\n', df[(df['days_employed'] < 21900) & (df['days_employed'] > 0)])

# корректируем отрицательные значения в стаже
counter = 0
for row in df['days_employed']:
    if df.loc[counter, 'days_employed'] < 0:
        df.loc[counter, 'days_employed'] = df.loc[counter, 'days_employed'] * (-1)
        counter += 1
    else:
        counter += 1
        continue
        
# добавляем колонку с временем стажа по периодам (можно без нее, но так проще на текущем этапе)
def years(row):
    if (row['days_employed'] / 365) == 0:
        df_dobs = df[df['dob_years'] == row['dob_years']]
        work_days = df_dobs['days_employed'].mean()
        if (work_days / 365) > 0 and (work_days / 365) < 1:
            return '1'
        if (work_days / 365) >= 1 and (work_days / 365) < 3:
            return '3'
        if (work_days / 365) >= 3 and (work_days / 365) < 5:
            return '5'
        if (work_days / 365) >= 5 and (work_days / 365) < 10:
            return '10'
        if (work_days / 365) >= 10:
            return '10+'
    if (row['days_employed'] / 365) > 0 and (row['days_employed'] / 365) < 1:
        return '1'
    if (row['days_employed'] / 365) >= 1 and (row['days_employed'] / 365) < 3:
        return '3'
    if (row['days_employed'] / 365) >= 3 and (row['days_employed'] / 365) < 5:
        return '5'
    if (row['days_employed'] / 365) >= 5 and (row['days_employed'] / 365) < 10:
        return '10'
    if (row['days_employed'] / 365) >= 10:
        return '10+'
    
df['work_years'] = df.apply(years, axis=1)

# далее идет очень нерациональный способ, но не понимаю, почему цикл не выполняется с первого раза и его приходится повторять еще несколько раз. Медианные значения берутся из массива, который не меняется, но из раза в раз Nan позиции убывают
data_2 = df['work_years']
counter = 0

for element in data_2:
    if (data_2.isna().values[counter] == True) == True:
        data_2_w_condition = df[df['dob_years'] == df.loc[counter, 'dob_years']]
        data_2_w_condition_mean = data_2_w_condition['days_employed'].mean()
        if (data_2_w_condition_mean / 365) > 0 and (data_2_w_condition_mean / 365) < 1:
            df.loc[counter, 'work_years'] = '1'
        if (data_2_w_condition_mean / 365) >= 1 and (data_2_w_condition_mean / 365) < 3:
            df.loc[counter, 'work_years'] = '3'
        if (data_2_w_condition_mean / 365) >= 3 and (data_2_w_condition_mean / 365) < 5:
            df.loc[counter, 'work_years'] = '5'
        if (data_2_w_condition_mean / 365) >= 5 and (data_2_w_condition_mean / 365) < 10:
            df.loc[counter, 'work_years'] = '10'
        if (data_2_w_condition_mean / 365) >= 10:
            df.loc[counter, 'work_years'] = '10+'
    else:
        counter += 1
        continue

print(df['work_years'].isna().sum())

data_3 = df['work_years']
counter = 0

for element in data_3:
    if (data_3.isna().values[counter] == True) == True:
        data_3_w_condition = df[df['dob_years'] == df.loc[counter, 'dob_years']]
        data_3_w_condition_mean = data_3_w_condition['days_employed'].mean()
        if (data_3_w_condition_mean / 365) > 0 and (data_3_w_condition_mean / 365) < 1:
            df.loc[counter, 'work_years'] = '1'
        if (data_3_w_condition_mean / 365) >= 1 and (data_3_w_condition_mean / 365) < 3:
            df.loc[counter, 'work_years'] = '3'
        if (data_3_w_condition_mean / 365) >= 3 and (data_3_w_condition_mean / 365) < 5:
            df.loc[counter, 'work_years'] = '5'
        if (data_3_w_condition_mean / 365) >= 5 and (data_3_w_condition_mean / 365) < 10:
            df.loc[counter, 'work_years'] = '10'
        if (data_3_w_condition_mean / 365) >= 10:
            df.loc[counter, 'work_years'] = '10+'
    else:
        counter += 1
        continue

print(df['work_years'].isna().sum())

data_5 = df['work_years']
counter = 0

for element in data_5:
    if (data_5.isna().values[counter] == True) == True:
        data_5_w_condition = df[df['dob_years'] == df.loc[counter, 'dob_years']]
        data_5_w_condition_mean = data_5_w_condition['days_employed'].mean()
        if (data_5_w_condition_mean / 365) > 0 and (data_5_w_condition_mean / 365) < 1:
            df.loc[counter, 'work_years'] = '1'
        if (data_5_w_condition_mean / 365) >= 1 and (data_5_w_condition_mean / 365) < 3:
            df.loc[counter, 'work_years'] = '3'
        if (data_5_w_condition_mean / 365) >= 3 and (data_5_w_condition_mean / 365) < 5:
            df.loc[counter, 'work_years'] = '5'
        if (data_5_w_condition_mean / 365) >= 5 and (data_5_w_condition_mean / 365) < 10:
            df.loc[counter, 'work_years'] = '10'
        if (data_5_w_condition_mean / 365) >= 10:
            df.loc[counter, 'work_years'] = '10+'
    else:
        counter += 1
        continue
        
print(df['work_years'].isna().sum())

data_6 = df['work_years']
counter = 0
for element in data_6:
    if (data_6.isna().values[counter] == True) == True:
        data_6_w_condition = df[df['dob_years'] == df.loc[counter, 'dob_years']]
        data_6_w_condition_mean = data_6_w_condition['days_employed'].mean()
        if (data_6_w_condition_mean / 365) > 0 and (data_6_w_condition_mean / 365) < 1:
            df.loc[counter, 'work_years'] = '1'
        if (data_6_w_condition_mean / 365) >= 1 and (data_6_w_condition_mean / 365) < 3:
            df.loc[counter, 'work_years'] = '3'
        if (data_6_w_condition_mean / 365) >= 3 and (data_6_w_condition_mean / 365) < 5:
            df.loc[counter, 'work_years'] = '5'
        if (data_6_w_condition_mean / 365) >= 5 and (data_6_w_condition_mean / 365) < 10:
            df.loc[counter, 'work_years'] = '10'
        if (data_6_w_condition_mean / 365) >= 10:
            df.loc[counter, 'work_years'] = '10+'
    else:
        counter += 1
        continue
        
print(df['work_years'].isna().sum())

# заполняем отсутсвующие значение медианными значениями, исходя из группы, к которой относится заёмщик
data = df['total_income']
counter = 0

for element in data:
    if (data.isna().values[counter] == True) == True:
        data_w_condition = df[(df['income_type'] == df.loc[counter, 'income_type']) & 
                              (df['gender'] == df.loc[counter, 'gender']) & 
                             (df['education_id'] == df.loc[counter, 'education_id']) & 
                             (df['work_years'] == df.loc[counter, 'work_years'])]
        data_w_condition_mean = data_w_condition['total_income'].mean()
        df.loc[counter, 'new_total_income'] = data_w_condition_mean # новый столбец, чтобы увеличивающееся количество медианных данных чтобы на каждый новый виток цикла не сказывались накапливающиеся медианные данные
        counter += 1
    else:
        df.loc[counter, 'new_total_income'] = df.loc[counter, 'total_income'] 
        counter += 1
        continue
    
# костыли...
data = df['new_total_income']

counter = 0
for element in data:
    if (data.isna().values[counter] == True) == True:
        print(counter)
        counter += 1
    else:
        counter +=1
        continue

a_1 = df[(df['gender'] == df.loc[8142, 'gender']) & (df['education_id'] == df.loc[8142, 'education_id']) & (df['work_years'] == df.loc[8142, 'work_years'])]
df.loc[8142,'new_total_income'] = a_1['total_income'].mean()

a_2 = df[(df['gender'] == df.loc[5936, 'gender']) & (df['education_id'] == df.loc[5936, 'education_id']) & (df['work_years'] == df.loc[5936, 'work_years'])]
df.loc[5936,'new_total_income'] = a_2['total_income'].mean()

a_3 = df[(df['gender'] == df.loc[21510, 'gender']) & (df['education_id'] == df.loc[21510, 'education_id'])]
df.loc[21510,'new_total_income'] = a_3['total_income'].mean()

print(df.isna().sum())

2173
212
17
1
5936
8142
21510
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
work_years             1
new_total_income       0
dtype: int64


Вывод

Имеются пропущенные данные по столбцам стажа и дохода.
Причиной пропущенных значений могло послужить:
     1. Отсутствие трудового стажа на момент получения данных
     2. Человеческий / Технический фактор по отсутствию заполненных данных
     3. Отсутвие требования по заполнению данных на момент сбора данных (первоначальная недоработанная версия анкеты)

За основу берем п.2. и заполняем данные медианными значениями 

Имеются отрицательные значения и чрезмерно высокие положительные значения по колонке "стаж".  
Положительные значения являются некорректными в связи с превышающим возможный стаж человека, их аннулируем, чтобы не вводить в заблуждение при последующих действиях с колонками
Отрицательные значения перевод в положительные, т.к. визуально соответствуют реальности. 

Заполняем пропущенные значения в рамках дохода медианными данными, исходя из категории, в которой относится кредитор (стаж, пол, образование, профессия). Берем за основу медианные, т.к. лучше отражает средний доход по категории, без влияния сверхбогатых и сверхбедных.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Пропуски заполнены, молодец)

### Замена типа данных

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
work_years          21524 non-null object
new_total_income    21525 non-null float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB
None


### Вывод

Я понимаю, что в критериях указано: "вещественный на целочисленный". Мы меняли данные в столбцах:
- стаж (дробное, но все еще число)
- доход (дробное, но все еще число)

Чисто визуально не нахожу некорректной вещественной колонки. Также при работе с данными далее нет проблем для анализа и подготовки выводов. Буду благодарен за помощь)

<div style="border:solid red 2px; padding: 20px"> <h1 style="color:red; margin-bottom:20px">Комментарий наставника</h1>

Стоило заменить типы данных для столбцов, в которых заполнял пропуски, для более удобной работы и в критериях принятия работы указано какой тип на какой менять

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Хорошо

### Обработка дубликатов

In [4]:
df = df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum())

0


### Вывод

Для поиска дубликатов используем метод duplicated() вместе с методом суммирования значений.
При удалении данных обнуляем индексы, чтобы не было пропущенных значений, которые вводят в заблуждение и приводят к ошибке в работе с ними.
Данные могли задублироваться по причинам пользователя: отправлена заявка больше одного раза(возможно из-за тех неполадок на сайте или отсутствия отклика на первичную заявку пользователя). Или по техническим причинам. 

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Дубликаты найдены и обработаны верно,отлично


### Лемматизация

In [5]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

# Создаем функцию для получения инфинитива каждой строки
def lemmatizator(row):
    return m.lemmatize(row['purpose'])

df['words_in_purpose'] = df.apply(lemmatizator, axis=1)

test_data = df.loc[:,'words_in_purpose']

list = []
for row in test_data:
    for word in row:
        list.append(word)
        
print(Counter(list))

# не во всех целях присутсвует типизация. К примеру, "недвижимость" (коммерческая? / жилая?). Поэтому список обобщенный
purposes = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']
purposes_data = df['words_in_purpose']
line = 0

for row in purposes_data:
    for word in row:
        if word in purposes:
            df.loc[line, 'general_purposes'] = word
            line += 1
        else:
            continue

df.loc[(df['general_purposes'] == 'жилье'), 'general_purposes'] = 'недвижимость'
print(df.isna().sum())

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})
children               0
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
work_years             1
new_total_income       0
words_in_purpose       0
general_purposes       0
dtype: int64


### Вывод

Т.к. нельзя лемматизировать весь Series 'purpose' целиком, то проводим лемматизацию на каждую строку и далее соединяем в один большой список, в рамках которого выделяем наиболее популярные цели кредита. Т.к. не везде присутствует уточняющие данные по цели кредита, то взяты общие категории: 'недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба'

В дальнейшем можно провести более глубокую лемматизацию по подтипам. К примеру, недвижимость: коммерческая / жилая, ремонт / покупка, стротельство недвижимости / покупка недвижимости и т.д. 

### Категоризация данных

### Вывод

Данные категоризированы в рамках выводов, согасно задаваемому вопросу.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>


Лемматизация и категоризация проведена верно

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [1]:
# Отбрасываем строки со значениям "20" в колонке "Количество детей", т.к. данные выглядят некорректными
df_children = df[(df['children'] != 20) & (df['children'] != -1)]

#Объединяем заемщиков, у которых есть дети
df_children.loc[(df_children['children'] > 0), 'children'] = 1

children_summary = df_children.pivot_table(index='children', values='debt')

print('Доля задолженностей по кредиту среди бездетных заемщиков: {:.2%}'.format(children_summary.loc[0,'debt']))
print('Доля задолженностей по кредиту среди заемщиков с детьми: {:.2%}'.format(children_summary.loc[1,'debt']))
print('На {:.2%} кредиторы с детьми чаще являются задолжниками по сравнению с бездетными'.format(children_summary.loc[1,'debt'] / children_summary.loc[0,'debt'] - 1))


NameError: name 'df' is not defined

# Вывод

Есть небольшая зависимость между кредиторами с детьми и без.
Кредиторы с детьми чаще являются задолжниками, нежели кредиторы без детей - на 22.5%

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [89]:
print(df.pivot_table(index='family_status', values='debt'))

                           debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093471
женат / замужем        0.075452


### Вывод

Также есть минимальная зависимость по семейному статусу. 
Так наименьшее количество задолжников среди кредиторов имеют статус "вдовец / вдова" - 6,5%
По сравнению с ними почти в половину больше невозвратных случае у кредиторов "не женат / замужем" и "гражданский брак", которые являются наибольшими задолжниками, срежи всех групп.

Если смотреть с точки зрения проживает один или с кем-то, то заивимость не наблюдается. Но можно отметить, что пользовати  прошедшие через институт брака или находящиеся в нем имеют лучший процент возврата средств. Возможно это связано с возрастом, необходимо проводить дополнительный анализ.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [91]:
def income_group(data):
    if data['new_total_income'] < 70000:
        return '0 < x < 70K'
    if (data['new_total_income'] >= 70000) & (data['new_total_income'] < 100000):
        return '70K <= x < 100K'
    if (data['new_total_income'] >= 100000) & (data['new_total_income'] < 150000):
        return '100K <= x < 150K'
    if (data['new_total_income'] >= 150000) & (data['new_total_income'] < 200000):
        return '150K <= x < 200K'
    if (data['new_total_income'] >= 200000) & (data['new_total_income'] < 300000):
        return '200K <= x < 300K'
    if (data['new_total_income'] >= 300000) & (data['new_total_income'] < 500000):
        return '300K <= x < 500K'
    if (data['new_total_income'] >= 500000) & (data['new_total_income'] < 1000000):
        return '500K <= x < 1M'
    if data['new_total_income'] >= 1000000:
        return '1M <= x'
    
df['income_group'] = df.apply(income_group, axis=1)

print(df.pivot_table(index='income_group', values='debt'))

                      debt
income_group              
0 < x < 70K       0.068475
100K <= x < 150K  0.085327
150K <= x < 200K  0.089604
1M <= x           0.080000
200K <= x < 300K  0.069528
300K <= x < 500K  0.072884
500K <= x < 1M    0.060914
70K <= x < 100K   0.084615


### Вывод

Зависимости между невозвратом кредита и доходов кредитора не наблюдается.
Можно отметить, что кредиторы с минимальными доходами и доходами от 200К до 300К д.е / месяц имеют статистику вовзратов лучше по группе с доходом до 500К д.е. / месяц. Остальные категории имеют минимальную разницу ~<10% между собой

Можно отметить, что после выхода из черты минимального дохода (до 70К/месяц) пользователи начинают с большей вероятностью становитсья должниками банка до момента получения дохода 200К+

- Как разные цели кредита влияют на его возврат в срок?

In [93]:
print(df.pivot_table(index='general_purposes', values='debt'))

                      debt
general_purposes          
автомобиль        0.093590
недвижимость      0.072334
образование       0.092200
свадьба           0.080034


### Вывод

Есть зависимость целей от возврата кредита.
Так наиболее ответствуенно относятся к возврату кредита пользователи, которые берут на действия с недвижимостью.Возможно большие суммы лучше влияют на ответственный подход пользователя.
Наибольшая доля задолжников у целей с автомобилем и образованием.

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Выводы все правильные, молодец, но задание лучше было бы выполнить используя сводные таблицы


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

В рамках работы были дозаполненые отсутствующие данные исходя из логики отношения пользователя к категории пользователей на основании медианных значений, как наиболе "справедливых" для оценки среднего значения.

Полученные данные были проанализированы и почти во всех случаях были выявлены минимальные расхождения между группами:
- Кредиторы с детьми чаще являются задолжниками по сравнению с бесдетными (+22.5%)
- Наибольшими должниками являются кредитозаемщики в статусе "не женат / не замужем". Особенность: пользовати прошедшие через институт брака или находящиеся в нем имеют лучший процент возврата средств.
- Закономерности между доходов и возвратом кредита не было выявлено. Но стоит отметить, что после выхода из черты минимального дохода (до 70К/месяц) пользователи начинают с большей вероятностью становитсья должниками банка до момента получения дохода 200К+
- Наиболее ответственными являются заемщики с целью операций с недвижимостью. Противоположность - операции с автомобилями и образованием.

Рекомендовано применить полученные результаты для составления скоринговой модели

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Обязательно надо написать подробный вывод из проведённого анализа, в котором отразить все полученные результаты + дать советы по использованию данного анализа


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Точные и обоснованные выводы, супер)

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Работа хорошая, но стоит исправить обозначенные неточности, буду ждать исправленный вариант, удачи!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Работа принята!) Очень рад, что у тебя всё получилось и надеюсь тебе понравится весь курс, будь усерден и у тебя всё получится! Удачи в будущих проектах)